In [1]:
faqs = """
What is Metridash?
Metridash is an all-in-one content creation platform designed to empower creators like you. It combines AI-driven content generation, seamless multichannel publishing, content management, and real-time analytics to streamline your content creation process.
How can Metridash help me as a creator?
Metridash helps creators overcome creative blocks by generating personalized content ideas and prompts. It simplifies content management across multiple platforms, saving you time and effort. Additionally, it provides real-time analytics to help you track content performance and make data-driven decisions.
Do I need to be tech-savvy to use Metridash?
Not at all! Metridash is designed with user-friendliness in mind. Whether you're a tech guru or a beginner, you'll find our platform intuitive and easy to navigate.
Can I use Metridash for different content platforms like YouTube, Instagram, and my TikTok?
Absolutely! Metridash supports content creation and publishing across a wide range of platforms, including YouTube, Instagram, Twitter, Facebook, TikTok, LinkedIn, and blogs. You can manage all your content in one place.
How does Metridash generate content ideas?
Metridash uses a proprietary algorithm that takes your niche, target audience, and other details into account to generate personalized content ideas and prompts. It's like having a creative brainstorming partner at your side.
Is my data safe with Metridash?
We take data security seriously. Metridash employs robust security measures to protect your data. You can refer to our Privacy Policy for more details on how we handle and safeguard your information.
How can I get started with Metridash?
Getting started is easy. Simply sign up for a Metridash account, and you'll gain access to a world of content creation possibilities. We offer both free and premium plans to suit your needs.
"""

In [2]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.fit_on_texts([faqs])

In [5]:
len(tokenizer.word_index)

156

In [6]:
input_sequences = []
for sentence in faqs.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [7]:
max_len = max([len(x) for x in input_sequences])

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences,maxlen = max_len, padding='pre')

In [9]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [10]:
X.shape

(275, 41)

In [11]:
y.shape

(275,)

In [12]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=157)

In [13]:
y.shape

(275, 157)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [15]:
model = Sequential()
model.add(Embedding(157,100,input_length=41))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(157,activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 41, 100)           15700     
                                                                 
 lstm (LSTM)                 (None, 41, 150)           150600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 157)               23707     
                                                                 
Total params: 370607 (1.41 MB)
Trainable params: 370607 (1.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.fit(X,y,epochs=200)

Epoch 1/200
9/9 [==============================] - 7s 160ms/step - loss: 5.0507 - accuracy: 0.0255
Epoch 2/200
9/9 [==============================] - 1s 142ms/step - loss: 4.9330 - accuracy: 0.0400
Epoch 3/200
9/9 [==============================] - 1s 88ms/step - loss: 4.7888 - accuracy: 0.0364
Epoch 4/200
9/9 [==============================] - 1s 131ms/step - loss: 4.7447 - accuracy: 0.0509
Epoch 5/200
9/9 [==============================] - 1s 93ms/step - loss: 4.7285 - accuracy: 0.0327
Epoch 6/200
9/9 [==============================] - 0s 47ms/step - loss: 4.6918 - accuracy: 0.0545
Epoch 7/200
9/9 [==============================] - 1s 69ms/step - loss: 4.6595 - accuracy: 0.0691
Epoch 8/200
9/9 [==============================] - 0s 11ms/step - loss: 4.6068 - accuracy: 0.0836
Epoch 9/200
9/9 [==============================] - 0s 29ms/step - loss: 4.5470 - accuracy: 0.0691
Epoch 10/200
9/9 [==============================] - 0s 29ms/step - loss: 4.4926 - accuracy: 0.0727
Epoch 11/200
9/9

In [19]:
import time
import numpy as np
text = "Metridash uses"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=41, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 1s 816ms/step
Metridash uses a
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm that
1/1 [==============================] - 0s 20ms/step
Metridash uses a proprietary algorithm that takes
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm that takes your
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm that takes your niche
1/1 [==============================] - 0s 18ms/step
Metridash uses a proprietary algorithm that takes your niche target
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm that takes your niche target audience
1/1 [==============================] - 0s 19ms/step
Metridash uses a proprietary algorithm 

In [20]:
model.save("metridash_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
